# APIs and FastAPI (part 1)

# APIs

## HTTP request methods (aka HTTP verbs)
(https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods)

### GET
- requests a representation of the specified resource
- only retrieves data

### POST
- used to submit an entity to specified resource
- often causes change or side effects on server

### PUT
- replaces all current representations of target resource with the request payload

### DELETE
- deletes the specified resource

### PATCH
- applies partial modifications to a resource

## HTTP Status Codes
(https://httpstatuses.com/)
- 1×× Informational
- 2×× Success
- 3×× Redirection
- 4×× Client Error
- 5×× Server Error

## JSON Schema
(https://json-schema.org/)
- vocabulary that allows you to annotate and validate JSON documents
- benefits:
    - describes existing format(s)
    - human-readable and machine-readable documentation
    - provides way to validate data
- a JSON Schema itself is written in JSON
- uses JSON types (string, number, object, array, boolean, null)

## Example

In [1]:
from jsonschema import validate

household_schema = {
    "type": "object",
    "properties": {
        "id": {"type": "number"},
        "street_address": {"type": "string"},
        "city": {"type": "string"},
        "state": {"type": "string"},
        "zipcode": {"type": "string"},
        "occupants": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": {"type": "string"},
                    "age": {"type": "number"}
                }
            }
        }
    }
}

In [2]:
household = {
    "id": 1,
    "street_address": "123 Oak St",
    "city": "Philadelphia",
    "state": "PA",
    "zipcode": "19147",
    "occupants": [
        {"name": "John Smith", "age": 25},
        {"name": "John Doe", "age": 27},
    ]
}
validate(instance=household, schema=household_schema)

In [3]:
household["id"] = "NAN"
validate(household, household_schema)

ValidationError: 'NAN' is not of type 'number'

Failed validating 'type' in schema['properties']['id']:
    {'type': 'number'}

On instance['id']:
    'NAN'

## OpenAPI Specification (OAS)
(https://swagger.io/specification/)
- formerly known as Swagger Specification; Swagger refers to a set of tools around OAS
- API description format for REST APIs
- describes:
    - endpoints (e.g. /users)
    - operations on each endpoint (e.g. GET /users)
    - authentication methods
    - contact info, license, etc.
- can be written in YAML or JSON

### Examples
- https://github.com/OAI/OpenAPI-Specification/tree/master/examples/v3.0

### Why use OAS?
- collaborate on API design
- codegen tool to generate code from spec (stubs for server-side logic, mock server, etc.)
- generation of interactive API documentation
- connect tools that are compatible with spec
- use for quality validation

# FastAPI

"modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints"
(https://fastapi.tiangolo.com/)

### Why use?
- very performant
- quick to code
- reduces human errors
- production-ready code w/ automatic documentation
- based on OpenAPI and JSON schema
- built using Starlette and Pydantic

## Starlette

"lightweight ASGI framework/toolkit, which is ideal for building high performance asyncio services"
(https://www.starlette.io/)

- WSGI
    - Web Server Gateway Interface
    - convention for web servers to forward requests to web apps or frameworks written in Python (PEP 3333)
    - example WSGI servers: Gunicorn, mod_wsgi, uWSGI

- ASGI
    - Asynchronous Server Gateway Interface
    - provides standard for asynchronous and synchronous apps
    - example ASGI servers: daphne, uvicorn, hypercorn

## Pydantic

"Data validation and settings management using python type annotations"
(https://pydantic-docs.helpmanual.io/)

In [4]:
# Example use from docs
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []


external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}
user = User(**external_data)
user.dict()

{'id': 123,
 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
 'friends': [1, 2, 3],
 'name': 'John Doe'}

In [5]:
# example error from docs
from pydantic import ValidationError

try:
    User(signup_ts='broken', friends=[1, 2, 'not number'])
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "id"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "signup_ts"
    ],
    "msg": "invalid datetime format",
    "type": "value_error.datetime"
  },
  {
    "loc": [
      "friends",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]


### Why use?
- simple, leverages existing type hints
- aids auto-completion, linting, etc. as type hints do
- can be used to validate request data and to load system settings
- works with complex structures: recursive models and custom validators
- can add custom data types to extend `typing`'s

In [6]:
# Recursive Example from docs
from typing import List
from pydantic import BaseModel


class Foo(BaseModel):
    count: int
    size: float = None


class Bar(BaseModel):
    apple = 'x'
    banana = 'y'


class Spam(BaseModel):
    foo: Foo
    bars: List[Bar]


m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])
print(m, "\n")
print(m.dict())

foo=Foo(count=4, size=None) bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')] 

{'foo': {'count': 4, 'size': None}, 'bars': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


In [7]:
# Validator Example from docs
from pydantic import BaseModel, ValidationError, validator


class UserModel(BaseModel):
    name: str
    username: str
    password1: str
    password2: str

    @validator('name')
    def name_must_contain_space(cls, v):
        if ' ' not in v:
            raise ValueError('must contain a space')
        return v.title()

    @validator('password2')
    def passwords_match(cls, v, values, **kwargs):
        if 'password1' in values and v != values['password1']:
            raise ValueError('passwords do not match')
        return v

    @validator('username')
    def username_alphanumeric(cls, v):
        assert v.isalnum(), 'must be alphanumeric'
        return v


user = UserModel(
    name='samuel colvin',
    username='scolvin',
    password1='zxcvbn',
    password2='zxcvbn',
)
print(user, "\n")
try:
    UserModel(
        name='samuel',
        username='scolvin',
        password1='zxcvbn',
        password2='zxcvbn2',
    )
except ValidationError as e:
    print(e)

name='Samuel Colvin' username='scolvin' password1='zxcvbn' password2='zxcvbn' 

2 validation errors for UserModel
name
  must contain a space (type=value_error)
password2
  passwords do not match (type=value_error)
